In [1]:
import pickle
import gzip
import numpy as np
import os

In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

## Visualization of images

In [ ]:
# load data
train_data = load_zipped_pickle("./data/train.pkl")
#test_real_data = load_zipped_pickle("./data/test.pkl")


In [ ]:
# Calculate the number of videos
num_videos = len(train_data)

# Calculate the percentage of videos that are amateur or expert
num_amateur = sum(data['dataset'] == 'amateur' for data in train_data)
num_expert = sum(data['dataset'] == 'expert' for data in train_data)
percentage_amateur = (num_amateur / num_videos) * 100
percentage_expert = (num_expert / num_videos) * 100

# Display the statistics
print(f"Number of videos: {num_videos}")
print(f"Percentage of videos that are amateur: {percentage_amateur:.3}%")
print(f"Percentage of videos that are expert: {percentage_expert:.3}%")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming train_data is a list containing your data
# Extract two samples, one labeled by an expert and one by an amateur
expert_video = next(item for item in train_data if item['dataset'] == 'expert')
amateur_video = next(item for item in train_data if item['dataset'] == 'amateur')

# Function to plot frames
def plot_frames(video, title):
    plt.figure(figsize=(12, 4))
    plt.suptitle(title)
    
    # Plot three labeled frames
    for i, frame_idx in enumerate(video['frames'][:3]):
        plt.subplot(1, 3, i + 1)
        plt.imshow(video['video'][:, :, frame_idx], cmap='gray')  # Assuming grayscale video
        plt.title(f'Frame {frame_idx}')
        plt.axis('off')

    plt.show()

# Plot frames for expert video
plot_frames(expert_video, 'Expert Labeled Video')

# Plot frames for amateur video
plot_frames(amateur_video, 'Amateur Labeled Video')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming train_data is a list containing your data
# Extract two samples, one labeled by an expert and one by an amateur
expert_video = next(item for item in train_data if item['dataset'] == 'expert')
amateur_video = next(item for item in train_data if item['dataset'] == 'amateur')

# Function to plot frames with labels
def plot_frames_with_labels(video, title_prefix):
    plt.figure(figsize=(12, 4))
    
    # Plot three labeled frames with labels displayed
    for i, frame_idx in enumerate(video['frames'][:3]):
        plt.subplot(1, 3, i + 1)
        frame = video['video'][:, :, frame_idx]
        label = video['label'][:, :, frame_idx]
        
        plt.imshow(frame, cmap='gray')  # Assuming grayscale video
        plt.title(f'{title_prefix} - Frame {frame_idx}')
        
        # Display labels on the image
        for h in range(label.shape[0]):
            for w in range(label.shape[1]):
                if label[h, w]:
                    plt.text(w, h, 'Label', color='red', fontsize=8, ha='center', va='center')

        plt.axis('off')

    plt.show()

# Plot frames for expert video with labels
plot_frames_with_labels(expert_video, f'Expert Labeled Video')

# Plot frames for amateur video with labels
plot_frames_with_labels(amateur_video, f'Amateur Labeled Video')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming train_data is a list containing your data
# Extract one sample for demonstration
sample_video = train_data[0]

# Function to plot frames with labels and boxes
def plot_frames_with_labels_and_boxes(video, title_prefix):
    plt.figure(figsize=(18, 6))
    
    # Plot three labeled frames with labels and boxes displayed
    for i, frame_idx in enumerate(video['frames'][:3]):
        plt.subplot(2, 3, i + 1)
        frame = video['video'][:, :, frame_idx]
        label = video['label'][:, :, frame_idx]
        box = video['box']
        
        plt.imshow(frame, cmap='gray')  # Assuming grayscale video
        plt.title(f'{title_prefix} - Frame {frame_idx}')
        
        # Display labels on the image
        for h in range(label.shape[0]):
            for w in range(label.shape[1]):
                if label[h, w]:
                    plt.text(w, h, 'Label', color='red', fontsize=8, ha='center', va='center')
        
        # Plot the bounding box
        plt.subplot(2, 3, i + 4)
        plt.imshow(frame, cmap='gray')  # Assuming grayscale video
        plt.title(f'{title_prefix} - Frame {frame_idx} with Box')
        
        # Display the bounding box
        box_indices = np.where(box)
        plt.plot([box_indices[1].min(), box_indices[1].max(), box_indices[1].max(), box_indices[1].min(), box_indices[1].min()],
                 [box_indices[0].min(), box_indices[0].min(), box_indices[0].max(), box_indices[0].max(), box_indices[0].min()],
                 color='blue', linewidth=2)
        
        plt.axis('off')

    plt.show()

# Plot frames for the sample video with labels and boxes
plot_frames_with_labels_and_boxes(sample_video, f'Sample Video ({sample_video["name"]})')


## Preprocessing

In [3]:
data = load_zipped_pickle("./data/train.pkl")

In [23]:
import cv2
import numpy as np
from skimage import exposure
from skimage import transform as tf



def preprocess_data(data, new_height, new_width):

    video = data['video']
    box = data['box']
    label = data['label']

    # Determine the minimum dimension among height and width
    min_dimension = min(video.shape[0], video.shape[1])

    # Resize video and box maintaining the aspect ratio
    aspect_ratio = video.shape[1] / video.shape[0]
    resized_width = new_width if aspect_ratio >= 1 else int(min_dimension * aspect_ratio)
    resized_height = new_height if aspect_ratio <= 1 else int(min_dimension / aspect_ratio)

    print("Resized dimensions:", (resized_width, resized_height))

    resized_video = cv2.resize(video.transpose(2, 0, 1), (resized_width, resized_height), interpolation=cv2.INTER_LINEAR).transpose(1, 2, 0)
    resized_box = cv2.resize(box.astype(np.uint8), (resized_width, resized_height), interpolation=cv2.INTER_NEAREST).astype(bool)

    # Normalize video and standardize box
    normalized_video = resized_video / 255.0  # Normalize to [0, 1]
    standardized_box = (resized_box - np.mean(resized_box)) / np.std(resized_box)

    # Histogram Equalization on video
    equalized_video = np.zeros_like(resized_video)
    for i in range(resized_video.shape[2]):
        equalized_video[..., i] = exposure.equalize_hist(resized_video[..., i])

    # Data augmentation
    augmented_video, augmented_box, augmented_label = augment_data(normalized_video, standardized_box, label)

    return augmented_video, augmented_box, augmented_label

def augment_data(video, box, label):
    # Define data augmentation parameters
    rotation_angle = 30
    scaling_range = (0.8, 1.2)
    translation_range = (10, 10)
    shearing_range = 0.2

    # Randomly apply data augmentation
    if np.random.rand() > 0.5:
        # Rotation
        angle = np.random.uniform(-rotation_angle, rotation_angle)
        video = tf.rotate(video, angle, mode='reflect')
        box = tf.rotate(box.astype(float), angle, mode='reflect')
        label = tf.rotate(label.astype(float), angle, mode='reflect')

    if np.random.rand() > 0.5:
        # Scaling
        scaling_factor = np.random.uniform(scaling_range[0], scaling_range[1])
        video = tf.rescale(video, scale=(scaling_factor, scaling_factor, 1), mode='reflect')
        box = tf.rescale(box.astype(float), scale=(scaling_factor, scaling_factor), mode='reflect')

        # Rescale each channel separately for label
        rescaled_label_channels = []
        for i in range(label.shape[2]):
            rescaled_label_channels.append(tf.rescale(label[..., i].astype(float), scale=(scaling_factor, scaling_factor), mode='reflect'))
        label = np.stack(rescaled_label_channels, axis=-1)

    if np.random.rand() > 0.5:
        # Translation
        translation = (np.random.uniform(-translation_range[0], translation_range[0]),
                       np.random.uniform(-translation_range[1], translation_range[1]))
        video = tf.warp(video, tf.AffineTransform(translation=translation), mode='reflect')
        box = tf.warp(box.astype(float), tf.AffineTransform(translation=translation), mode='reflect')

        # Translate each channel separately for label
        translated_label_channels = []
        for i in range(label.shape[2]):
            translated_label_channels.append(tf.warp(label[..., i].astype(float), tf.AffineTransform(translation=translation), mode='reflect'))
        label = np.stack(translated_label_channels, axis=-1)

    if np.random.rand() > 0.5:
        # Shearing
        shearing_factor = np.random.uniform(-shearing_range, shearing_range)
        video = tf.warp(video, tf.AffineTransform(shear=shearing_factor), mode='reflect')
        box = tf.warp(box.astype(float), tf.AffineTransform(shear=shearing_factor), mode='reflect')

        # Shear each channel separately for label
        sheared_label_channels = []
        for i in range(label.shape[2]):
            sheared_label_channels.append(tf.warp(label[..., i].astype(float), tf.AffineTransform(shear=shearing_factor), mode='reflect'))
        label = np.stack(sheared_label_channels, axis=-1)

    return video, box, label

In [24]:
new_height, new_width = 500, 500 

preprocessed_videos = []
preprocessed_boxes = []
preprocessed_labels = []

for sample in data:
    preprocessed_video, preprocessed_box, preprocessed_label = preprocess_data(sample, new_height, new_width)
    
    preprocessed_videos.append(preprocessed_video)
    preprocessed_boxes.append(preprocessed_box)
    preprocessed_labels.append(preprocessed_label)

# Convert lists to numpy arrays for further processing
preprocessed_videos = np.array(preprocessed_videos)
preprocessed_boxes = np.array(preprocessed_boxes)
preprocessed_labels = np.array(preprocessed_labels)



Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized dimensions: (500, 500)
Resized 

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:3699: error: (-215:Assertion failed) !dsize.empty() in function 'cv::hal::resize'


In [60]:
print(len(preprocessed_video))

NameError: name 'preprocessed_video' is not defined

## U-net single channel

In [7]:
def build_unet(input_shape):
    inputs = keras.Input(shape=input_shape)
    # ... (U-Net architecture with appropriate layers)
    # Example U-Net architecture with contracting and expansive paths
    x = layers.Conv2D(32, 3, activation="relu", padding="same")(inputs)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)

    # Upsampling and concatenation
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.concatenate([x, layers.Conv2DTranspose(128, 3, activation="relu", padding="same")(layers.MaxPooling2D(pool_size=(2, 2))(x))])
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)

    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.concatenate([x, layers.Conv2DTranspose(64, 3, activation="relu", padding="same")(layers.MaxPooling2D(pool_size=(2, 2))(x))])
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)

    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.concatenate([x, layers.Conv2DTranspose(32, 3, activation="relu", padding="same")(layers.MaxPooling2D(pool_size=(2, 2))(x))])
    x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)

    outputs = layers.Conv2D(1, 1, activation="sigmoid")(x)

    return keras.Model(inputs=inputs, outputs=outputs)

In [8]:
# Build and train the model
model = build_unet(input_shape=frames.shape[1:])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_dataset, epochs=10, validation_data=val_dataset)  # Adjust epochs based on performance

# Save the trained model
model.save("mitral_valve_segmentation.h5")

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

NameError: name 'frames' is not defined

In [9]:
def jaccard_index(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return (intersection + 1e-7) / (union + 1e-7)


In [10]:
predictions = model.predict(test_dataset)

# Calculate Jaccard Index for each frame
jaccard_indices = []
for i, (frames, label, prediction) in enumerate(zip(test_dataset)):
    for frame, pred, gt in zip(frames, prediction, label):
        jaccard_index_value = jaccard_index(gt, pred)
        jaccard_indices.append(jaccard_index_value)

# Compute median Jaccard Index
median_jaccard_index = np.median(jaccard_indices)

print(f"Median Jaccard Index: {median_jaccard_index:.4f}")

NameError: name 'model' is not defined